# Assignment 4.1: Feature Store - Exercise
Sinthuja Bates

In [8]:
import time
import os
from sagemaker import get_execution_role, session
import boto3

region = boto3.Session().region_name
role = get_execution_role()
sm_client = boto3.client('sagemaker', region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## Part 1: Set Up Model Group

In [33]:
model_package_group_name = 'xgboost-breast-cancer-detection6'
model_package_group_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageGroupDescription" : "Sample model package group",
}

response = sm_client.create_model_package_group(**model_package_group_input_dict)
print('ModelPackageGroupArn: {}'.format(response['ModelPackageGroupArn']))

ModelPackageGroupArn: arn:aws:sagemaker:us-east-1:711667138246:model-package-group/xgboost-breast-cancer-detection6


In [32]:
description = sm_client.describe_model_package_group(
    ModelPackageGroupName=model_package_group_name
)
print(description)

{'ModelPackageGroupName': 'xgboost-breast-cancer-detection5', 'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:711667138246:model-package-group/xgboost-breast-cancer-detection5', 'ModelPackageGroupDescription': 'Sample model package group', 'CreationTime': datetime.datetime(2024, 6, 1, 17, 37, 55, 935000, tzinfo=tzlocal()), 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:711667138246:user-profile/d-bnyjs4fnucfp/snagalingam', 'UserProfileName': 'snagalingam', 'DomainId': 'd-bnyjs4fnucfp', 'IamIdentity': {'Arn': 'arn:aws:sts::711667138246:assumed-role/LabRole/SageMaker', 'PrincipalId': 'AROA2LMVKX3DIVFDWDQ6T:SageMaker'}}, 'ModelPackageGroupStatus': 'Completed', 'ResponseMetadata': {'RequestId': 'a19b75d3-3004-4474-b964-74b148aefdca', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a19b75d3-3004-4474-b964-74b148aefdca', 'content-type': 'application/x-amz-json-1.1', 'content-length': '612', 'date': 'Sat, 01 Jun 2024 17:38:09 GMT'}, 'RetryAttempts': 0}}


## Part 2: Set Up Model Package

In [34]:
image = '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1'
model_artifact = 's3://sagemaker-us-east-1-711667138246/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-06-01-16-26-21/xgb-2024-06-01-16-26-21/output/model.tar.gz'

response = sm_client.create_model_package(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageDescription='Model package for breast cancer detection using XGBoost',
    InferenceSpecification={
        'Containers': [{
            'Image': image,
            'ModelDataUrl': model_artifact,
            'Environment': {
                'SAGEMAKER_SUBMIT_DIRECTORY': "s3://sagemaker-us-east-1-711667138246/model-scripts/",
                'SAGEMAKER_PROGRAM': 'inference.py',
                'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
                'SAGEMAKER_REGION': 'us-east-1'
            }
        }],
        'SupportedTransformInstanceTypes': ['ml.m5.large'],
        'SupportedRealtimeInferenceInstanceTypes': ['ml.m5.large'],
        'SupportedContentTypes': ['text/csv'],
        'SupportedResponseMIMETypes': ['text/csv']
    }
)

print('ModelPackageArn: {}'.format(response['ModelPackageArn']))

ModelPackageArn: arn:aws:sagemaker:us-east-1:711667138246:model-package/xgboost-breast-cancer-detection6/1


In [35]:
model_package_arn = response['ModelPackageArn']

description = sm_client.describe_model_package(
    ModelPackageName=model_package_arn
)
print(description)

{'ModelPackageGroupName': 'xgboost-breast-cancer-detection6', 'ModelPackageVersion': 1, 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:711667138246:model-package/xgboost-breast-cancer-detection6/1', 'ModelPackageDescription': 'Model package for breast cancer detection using XGBoost', 'CreationTime': datetime.datetime(2024, 6, 1, 17, 40, 9, 957000, tzinfo=tzlocal()), 'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1', 'ImageDigest': 'sha256:cf81520a3b695293022793e292cf8bc3732b79231a6ebe1fb308086f6163a875', 'ModelDataUrl': 's3://sagemaker-us-east-1-711667138246/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-06-01-16-26-21/xgb-2024-06-01-16-26-21/output/model.tar.gz', 'Environment': {'SAGEMAKER_CONTAINER_LOG_LEVEL': '20', 'SAGEMAKER_PROGRAM': 'inference.py', 'SAGEMAKER_REGION': 'us-east-1', 'SAGEMAKER_SUBMIT_DIRECTORY': 's3://sagemaker-us-east-1-711667138246/model-scripts/'}}], 'SupportedTransformIns

## Part 3: Write the Model Card

In [55]:
# Define the model card content as a dictionary
model_card_content = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "$id": "http://json-schema.org/draft-07/schema#",
    "title": "SageMakerModelCardSchema",
    "description": "Default model card schema",
    "version": "0.1.0",
    "type": "object"
}

# Serialize the dictionary to a JSON string
content_string = json.dumps(model_card_content)
            
              

# Call to create the model card
response =  sm_client.create_model_card(
    ModelCardName='example-model-card',
    ModelCardStatus='Draft', 
    Content="",
    Tags=[
        {
            'Key': 'Project',
            'Value': 'Example Project'
        },
    ]
)

# Print the response from the API call
print(response)

{'ModelCardArn': 'arn:aws:sagemaker:us-east-1:711667138246:model-card/example-model-card', 'ResponseMetadata': {'RequestId': 'b0937f5e-ea3a-4cb1-b6ab-07b32b64118e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b0937f5e-ea3a-4cb1-b6ab-07b32b64118e', 'content-type': 'application/x-amz-json-1.1', 'content-length': '89', 'date': 'Sat, 01 Jun 2024 17:56:47 GMT'}, 'RetryAttempts': 0}}


In [57]:
model_card_arn = response['ModelCardArn']

description = sm_client.describe_model_card(
    ModelCardName=model_card_arn
)
print(description)

{'ModelCardArn': 'arn:aws:sagemaker:us-east-1:711667138246:model-card/example-model-card', 'ModelCardName': 'example-model-card', 'ModelCardVersion': 1, 'Content': '', 'ModelCardStatus': 'Draft', 'CreationTime': datetime.datetime(2024, 6, 1, 17, 56, 47, 355000, tzinfo=tzlocal()), 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:711667138246:user-profile/d-bnyjs4fnucfp/snagalingam', 'UserProfileName': 'snagalingam', 'DomainId': 'd-bnyjs4fnucfp'}, 'LastModifiedTime': datetime.datetime(2024, 6, 1, 17, 56, 47, 355000, tzinfo=tzlocal()), 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:711667138246:user-profile/d-bnyjs4fnucfp/snagalingam', 'UserProfileName': 'snagalingam', 'DomainId': 'd-bnyjs4fnucfp'}, 'ResponseMetadata': {'RequestId': 'b5f97238-2892-4ace-a44f-a5e5583b0fc4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b5f97238-2892-4ace-a44f-a5e5583b0fc4', 'content-type': 'application/x-amz-json-1.1', 'content-length': '607', 'date': 'Sat, 01 Ju